Imports

In [232]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

target = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv',index_col='PassengerId')
train.loc[pd.isna(train["HomePlanet"]), :].index

Index(['0064_02', '0119_01', '0210_01', '0242_01', '0251_01', '0303_01',
       '0315_01', '0321_01', '0382_01', '0402_01',
       ...
       '8938_01', '8961_04', '9030_02', '9043_01', '9072_01', '9084_01',
       '9194_01', '9248_01', '9257_01', '9274_01'],
      dtype='object', name='PassengerId', length=201)

MODEL

In [2]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input=12, n_hidden=24, n_output=1) -> None:
        super().__init__()

        self.linear_1 = nn.Linear(n_input, n_hidden)
        self.linear_2 = nn.Linear(n_input, n_output)

        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()

    def forward(self,x):
        x = self.linear_1(x)
        x = self.sigmoid(x)
        x = self.linear_2(x)
        x = self.softmax(x)
        return x


Dataset

In [257]:
class PassangerDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.y = torch.tensor(dataframe['Transported'].values)

        x_float32 = dataframe.drop('Transported', axis=1).values.astype('float32')

        self.x = torch.tensor(x_float32,dtype=torch.float32).view(-1,x_float32.shape[1])

        print(self.x.shape)
        min_vals = torch.min(self.x,dim=0).values
        max_vals = torch.max(self.x,dim=0)
        print(f'min: {min_vals}')

        normalized_matrix = self.x.clone()  # Create a copy to avoid modifying the original tensor
        normalized_matrix = (normalized_matrix - min_vals) / (max_vals - min_vals)
        print(f'NORMALIZED: {normalized_matrix}')


    def __getitem__(self, index):
        return self.x[index],self.y[index]

    def __len__(self):
        return len(self.dataframe)

Data cleaning

In [258]:


Y_train = train['Transported']
X_train = train.drop(columns=['Name']).dropna()
X_train[['Cabin_1','Cabin_2','Cabin_3']] = X_train['Cabin'].str.split('/').apply(pd.Series)
X_train = X_train.drop(columns='Cabin')
X_train = pd.get_dummies(X_train,columns=['HomePlanet','Destination','Cabin_1','Cabin_3'])


X_train.head()

dataset = PassangerDataset(X_train)
dataloader = DataLoader(dataset)


# for sample in dataloader:

#     print(f'data: {sample[0]}, target: {sample[1]}')


torch.Size([6764, 25])
min: torch.return_types.min(
values=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]),
indices=tensor([ 0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  5,  0,  1,  0,
         0,  0,  0,  0,  0,  1,  0]))


TypeError: unsupported operand type(s) for -: 'Tensor' and 'torch.return_types.min'

In [27]:
train.drop('Transported',axis=1)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
PassengerId,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther
9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley
9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon
